In [1]:
from queue import Queue
from threading import Thread
from typing import Dict, List
from PIL import Image
import cv2
import numpy as np

import face_tool
from face_model import FaceModel
from fps_metric import FPS
from scrfd import SCRFD
import time
from utils import check_in_out, get_data, maybe_create_folder, open_cam_usb
from web_cam import WebcamVideoStream
import glob

/Users/dominhtuan/anaconda3/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [2]:
face_detector = SCRFD(model_file='/Users/dominhtuan/Downloads/AnNinhSoiChieu/scrfd_10g_bnkps.onnx')
face_model = FaceModel(onnx_model_path='/Users/dominhtuan/Downloads/AnNinhSoiChieu/webface_r50.onnx')

width = 1280
height = 720
WINDOW_NAME = 'CameraCheckIn'

/Users/dominhtuan/anaconda3/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [3]:
import face_tool
def get_vector_img(image,theshold_w=0.4,theshold_h=0.2):
    face_boxes, kpss = face_detector.detect(image,theshold_w =theshold_w,theshold_h=theshold_h,thresh=0.2)
    if face_boxes is not None:
        face_boxes = face_boxes.astype(np.int32)
        face_boxes = face_boxes[:, :4]
        face_boxes = np.maximum(face_boxes, 0)
    face_list = []
    if face_boxes is not None:
        for (x1, y1, x2, y2) in face_boxes:
            face_list.append((x1, y1, x2, y2))
        face_encoding = face_tool.face_encoding(image=image, kpss=kpss)
    
        return face_list,face_encoding
    return None,None

In [ ]:
!pip3 install deep

In [4]:
image_cc = cv2.imread("/Users/dominhtuan/Downloads/anhcccdtinh.jpg")
face_list,face_encoding_cc = get_vector_img(image_cc,theshold_w=0.05,theshold_h=0.05)

In [5]:
def recognize_face(                                                                    
    image: np.ndarray,                                                                 
    kpss: np.ndarray,                                                                  
    known_faces: List[np.ndarray],                                                                                                                       
) -> List[str]:                                                                        
    face_encodings = face_tool.face_encoding(image=image, kpss=kpss)                   
    face_names = []                                                                    
                                                                                       
    # Face recognition                                                                 
    for face_encoding in face_encodings:                                               
        matches = face_tool.compare_faces(known_faces, face_encoding, tolerance=0.6)   
        face_distances = face_tool.face_distance(known_faces, face_encoding)                                   
        best_idx = np.argmin(face_distances)                                           
        name = "Unknown"                                                               
        if matches[best_idx]:                                                          
            name = "MR.KI"                                                
                                                                      
        if name == "Unknown":                                                          
            face_names.append("Unknown")                                               
        else:                                                                                    
            face_names.append(name)                                                                                   
    return face_names  

In [6]:
def worker(input_q: Queue, output_q: Queue,known_faces):
    while True:
        frame, kpss = input_q.get()
        names = recognize_face(     
                image=frame,            
                kpss=kpss,              
                known_faces=known_faces
            )                           
        output = {"names": names}
        output_q.put(output)

In [7]:
def start_app(video_capture):
    # Load biometric data
    known_faces = []
    name_faces = []
    face_dict = {}
    # known_faces, name_faces, face_dicict = load_data(known_faces, name_faces, face_dict)

    input_q = Queue(2)  # fps is better if queue is higher but then more lags
    output_q = Queue()
    for i in range(1):
        t = Thread(target=worker, args=(input_q, output_q,face_encoding_cc))
        t.daemon = True
        t.start()

    fps = FPS().start()
    pTime = 0
    frame_index = 0
    face_step =30
    detect_step = 2
    reset_step = 120
    r = {}
    checked_name = {}
    name = "Unknown"
    while True:
        frame_index += 1
        _, frame = video_capture.read()
        frame = cv2.flip(frame, 1)
        face_list = []
        if frame_index % detect_step == 0:
            face_boxes, kpss = face_detector.detect(frame)
            if face_boxes is not None:
                face_boxes = face_boxes.astype(np.int32)
                face_boxes = face_boxes[:, :4]
                face_boxes = np.maximum(face_boxes, 0)
                for (x1, y1, x2, y2) in face_boxes:
                    face_list.append((x1, y1, x2, y2))
                    # pass

            if kpss is not None and frame_index % face_step == 0:
                if not input_q.full():
                    input_q.put((frame, kpss))

        if output_q.empty():
            pass  # fill up queue
        else:
           pass
        if frame_index % reset_step == 0:
            name = "Unknown"

        if frame_index % (reset_step * 10) == 0:
            checked_name = {}
            frame_index = 0

        font = cv2.FONT_HERSHEY_COMPLEX

        for (x1, y1, x2, y2) in face_list:
            top = y1
            bottom = y2
            left = x1
            right = x2
            if name != "Unknown":
                checked_name[name] = True
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, "{}".format(name), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            else:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                if name == "Unknown":
                    cv2.putText(frame, "Unknown", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                else:
                    cv2.putText(frame, "{}".format(name), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        if r.get('success', False) and name != "Unknown":
            checked_name[name] = True
            cv2.putText(frame, "Da Diem Danh - {}".format(name), (50, 50), font, 1.5, (0, 255, 0), 2)
        else:
            if name == "Unknown":
                cv2.putText(frame, "Unknown", (50, 50), font, 1.5, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Da Diem Danh - {}".format(name), (50, 50), font, 1.5, (0, 0, 255), 2)
        
        cTime = time.time()
        fps_new = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(frame, f'FPS: {int(fps_new)}', (20,70), cv2.FONT_HERSHEY_PLAIN, 3 , (0,255,0),2)
        
        cv2.imshow(WINDOW_NAME, frame)
        cv2.moveWindow(WINDOW_NAME, 0, 0)
        fps.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    fps.stop()

In [8]:
def open_window(width, height):
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(WINDOW_NAME, width, height)
    cv2.moveWindow(WINDOW_NAME, 0, 0)
    cv2.setWindowTitle(WINDOW_NAME, 'Camera Demo for Jetson Nano')

In [9]:
video_capture = open_cam_usb(1)
open_window(width, height)

start_app(video_capture)

video_capture.release()
cv2.destroyAllWindows()